# Árvores II - Tarefa 2

### 1. Carregar as bases

Vamos carregar as bases lidas na tarefa passada. Se você salvou essas bases em arquivo texto, basta fazer a leitura com o comando ```pd.read_csv``` das seguintes bases:

- X_train
- Y_train
- X_test
- Y_test

Não se esqueça de considerar a leitura dos índices dos arquivos no ```read_csv()```!

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import graphviz

In [2]:
# Carregando as bases X_train, y_train, X_test e y_test
X_train = pd.read_csv('X_train_processed.csv', index_col=[0,1])
y_train = pd.read_csv('UCI HAR Dataset/train/y_train.txt', header=None, sep='\s+').iloc[:,0]

X_test = pd.read_csv('X_test_processed.csv', index_col=[0, 1])
y_test = pd.read_csv('UCI HAR Dataset/test/y_test.txt', header=None, sep='\s+').iloc[:, 0]

### 2. Divisão da base em Treino, Validação e Teste

A base já se encontra dividida em Treino e Validação. O que vamos fazer então é extrair uma base de Validação da base de Treino.

Extraia 25% da base de treino como base de validação.

In [3]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=23)

### 3. Melhores 3 variáveis

Rode uma árvore com profundidade máxima igual a 4 para prever a atividade humana com todas as variáveis.
Observe a importância das variáveis e considere as 3 variáveis com maior importância para os próximos passos.
Dica: utilize o atributo ```clf.feature_importances_``` da árvore treinada.

In [4]:
# Criando o modelo de árvore de decisão e ajustando-o aos dados de treinamento
modelo = DecisionTreeClassifier(max_depth=4, random_state=23).fit(X_train, y_train)

# Obtendo a importância das variáveis
importances = modelo.feature_importances_

# Criando um dataframe com as variáveis e suas importâncias
importances_df = pd.DataFrame({'Variavel': X_train.columns, 'Importancia': importances})

# Ordenando o dataframe pela importância de forma decrescente
importances_df = importances_df.sort_values(by='Importancia', ascending=False)                              

# Selecionando as 3 variáveis com maior importância
melhores_variaveis = importances_df.head(3)

print(melhores_variaveis)

                          Variavel  Importancia
52     (53, 'tGravityAcc-min()-X')     0.283604
271      (272, 'fBodyAcc-mad()-X')     0.254843
559  (560, 'angle(Y,gravityMean)')     0.168656


### 4. Construa uma árvore com as 3 melhores variáveis

Utilizando as três variáveis encontradas acima, construa uma árvore de decisão. Encontre o melhor ```ccp_alpha``` utilizando a base de validação, conforme a estrutura que vimos em aula.

In [5]:
%%time
X_train_best = X_train[melhores_variaveis['Variavel']]
X_val_best = X_val[melhores_variaveis['Variavel']]
# Selecionando apenas as três melhores variáveis


# Criando o modelo de árvore de decisão
modelo_best = DecisionTreeClassifier(random_state=23)

# Calculando os ccp_alphas
path = modelo_best.cost_complexity_pruning_path(X_train_best, y_train)
ccp_alphas, impurities = path.ccp_alphas, path.impurities

# Selecionando apenas alguns valores de ccp_alpha
ccp_alphas = ccp_alphas[::5]

# Ajustando o modelo aos dados de treinamento e validação para cada valor de ccp_alpha
melhores_modelos = []
for ccp_alpha in ccp_alphas:
    modelo = DecisionTreeClassifier(random_state=23, ccp_alpha=ccp_alpha)
    modelo.fit(X_train_best, y_train)
    melhores_modelos.append(modelo)

# Avaliando a acurácia de cada modelo na base de validação
val_acuracia = [modelo.score(X_val_best, y_val) for modelo in melhores_modelos]

# Encontrando o melhor ccp_alpha com base na acurácia de validação
melhor_ccp_alpha = ccp_alphas[np.argmax(val_acuracia)]
print("Melhor ccp_alpha:", melhor_ccp_alpha)

# Visualizando a árvore de decisão
modelo_final = DecisionTreeClassifier(random_state=23, ccp_alpha=melhor_ccp_alpha, max_depth=4)
modelo_final.fit(X_train_best, y_train)

dot_data = export_graphviz(
    modelo_final,
    out_file=None,
    feature_names=melhores_variaveis['Variavel'].tolist(),
    class_names=['1', '2', '3', '4', '5', '6'],
    filled=True,
    rounded=True,
    special_characters=True
)

graph = graphviz.Source(dot_data)
graph.render('arvore_decisao', view=True)


Melhor ccp_alpha: 0.00032471457933917145
CPU times: user 1.37 s, sys: 49.9 ms, total: 1.42 s
Wall time: 1.84 s


'arvore_decisao.pdf'

### 5. Avaliação do modelo

Avalie a árvore encontrada no item anterior na base de testes.

In [6]:
# Selecionando apenas as três melhores variáveis
X_test_best = X_test[melhores_variaveis['Variavel']]

# Criando o modelo de árvore de decisão com o melhor ccp_alpha encontrado na validação
modelo_final = DecisionTreeClassifier(random_state=23, ccp_alpha=melhor_ccp_alpha)

# Ajustando o modelo aos dados de treinamento
modelo_final.fit(X_train_best, y_train)

# Avaliando a acurácia do modelo na base de testes
test_auracia = modelo_final.score(X_test_best, y_test)
print("Acurácia na base de testes:", test_auracia)


Acurácia na base de testes: 0.7336274177129284
